# Infos

source
- https://www.geeksforgeeks.org/python-find-current-weather-of-any-city-using-openweathermap-api/
- https://openweathermap.org/

# import packages

In [1]:
import datetime as dt
import matplotlib.pyplot as plt
import csv
%matplotlib inline
import requests, json 
import os
from  myfunc import get_api_data, weather_data_call,format_current
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# import API Key

In [2]:
# use self made func
data=get_api_data("api_data/info.txt")
#arrange data
api_key=data[0][:-1]
base_url_current=data[1][:-1]
#base_url_forcast=data[2][:-1]
city_name=data[3][:-1]

# Get Data via API Call

In [3]:
request=weather_data_call(api_key,base_url_current,city_name)

In [7]:
#see result
#request.content
request.json()

{'coord': {'lon': 13.41, 'lat': 52.52},
 'weather': [{'id': 801,
   'main': 'Clouds',
   'description': 'few clouds',
   'icon': '02d'}],
 'base': 'stations',
 'main': {'temp': 280.39,
  'feels_like': 277.3,
  'temp_min': 278.71,
  'temp_max': 282.04,
  'pressure': 1025,
  'humidity': 48},
 'visibility': 10000,
 'wind': {'speed': 1},
 'clouds': {'all': 20},
 'dt': 1584183923,
 'sys': {'type': 1,
  'id': 1275,
  'country': 'DE',
  'sunrise': 1584163345,
  'sunset': 1584205720},
 'timezone': 3600,
 'id': 2950159,
 'name': 'Berlin',
 'cod': 200}

# Export as CSV

## file name

In [5]:
export_file_name='current/current_weater_berlin.csv'

## header

In [6]:
header=["datetime","temperature","humidity","pressure",
        "windspeed","winddir","cloudcoverage","weather",
        "description","weathericon","sunrisetime","sunsettime"]
len(header)

12

## data point

In [10]:
data_point_value=format_current(request)
data_point_value

KeyError: 'deg'

In [8]:
try:
    data_point_value=format_current(request)
    data_point=dict(zip(header, data_point_value))
    data_point
    
except:
    data_point_value=[]
    for i in range(len(header)):
        data_point_value.append('open weather API error')
    print(data_point_value)


['open weather API error', 'open weather API error', 'open weather API error', 'open weather API error', 'open weather API error', 'open weather API error', 'open weather API error', 'open weather API error', 'open weather API error', 'open weather API error', 'open weather API error', 'open weather API error']


## append to file

In [8]:
#append to csv file
with open(export_file_name, 'a', newline='') as f:
    header=["datetime","temperature","humidity","pressure","windspeed","winddir","cloudcoverage","weather","description","weathericon","sunrisetime","sunsettime"]
    writer = csv.DictWriter(f,delimiter=',',fieldnames=header)
    if os.path.getsize(export_file_name)==0:
           writer.writeheader()          
    writer.writerow(data_point)

# create Json database

# google API

In [35]:
#authentication to google api using JSON credential file
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('api_data/WDcred.json', scope)
client = gspread.authorize(creds)

In [36]:
#get current weather berlin sheet
current_weather = client.open('current_weather_berlin')

In [37]:
#append row

In [38]:
current_weather.sheet1.append_row(data_point_value)

{'spreadsheetId': '1Us3ubvtweNTf6NncVXHeW6XX5DeeJ8D4lvAfP6le1GQ',
 'tableRange': 'current_weater_berlin!A1:L14',
 'updates': {'spreadsheetId': '1Us3ubvtweNTf6NncVXHeW6XX5DeeJ8D4lvAfP6le1GQ',
  'updatedRange': 'current_weater_berlin!A15:L15',
  'updatedRows': 1,
  'updatedColumns': 12,
  'updatedCells': 12}}